<a href="https://colab.research.google.com/github/Sujata005/discord/blob/master/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import os
os.chdir("/content/drive/My Drive")

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving intents.json to intents (2).json


In [ ]:
data=next(iter(uploaded.values()))
type(data)

bytes

In [ ]:
import json
import pickle
import random
import csv
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer  # stem down the word for example works,working, work etc
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD


lemmatizer=WordNetLemmatizer()
intents= json.loads(data.decode())     #converts json file into dictionary


def train_model():
  words=[]
  classes=[]
  documents=[]
  ignore_letters=['?','!',',','.']

  for intent in intents['intents']:
      for pattern in intent['patterns']:
          word_list=nltk.word_tokenize(pattern)
          words.extend(word_list)
          documents.append((word_list,intent['tag']))
          if intent['tag'] not in classes:
              classes.append(intent['tag'])

  words=[lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
  words=sorted(set(words))

  classes=sorted(set(classes))
  pickle.dump(words, open('words.pkl','wb'))
  pickle.dump(classes, open('classes.pkl','wb'))

  training=[]
  output_empty=[0]* len(classes)

  for document in documents:
    bag=[]
    word_patterns=document[0]
    word_patterns=[lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
      bag.append(1) if word in word_patterns else bag.append(0)

    output_row=list(output_empty)
    output_row[classes.index(document[1])]=1
    training.append([bag, output_row])

  random.shuffle(training)
  training=np.array(training)

  train_x=list(training[:, 0])
  train_y=list(training[:, 1])

  model=Sequential()
  model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(len(train_y[0]), activation='softmax'))

  sgd=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
  model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
  hist=model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
  model.save('chatbot_model.hs',hist)
  print('done')

train_model()



Epoch 1/200


<ipython-input-8-54881eb673d0>:53: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training=np.array(training)
/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


6/6 [==============================] - 0s 3ms/step - loss: 1.5734 - accuracy: 0.2143
Epoch 2/200
6/6 [==============================] - 0s 3ms/step - loss: 1.5949 - accuracy: 0.2857
Epoch 3/200
6/6 [==============================] - 0s 3ms/step - loss: 1.5806 - accuracy: 0.5000
Epoch 4/200
6/6 [==============================] - 0s 3ms/step - loss: 1.5305 - accuracy: 0.2500
Epoch 5/200
6/6 [==============================] - 0s 3ms/step - loss: 1.4407 - accuracy: 0.4643
Epoch 6/200
6/6 [==============================] - 0s 3ms/step - loss: 1.5132 - accuracy: 0.3929
Epoch 7/200
6/6 [==============================] - 0s 3ms/step - loss: 1.4136 - accuracy: 0.5357
Epoch 8/200
6/6 [==============================] - 0s 3ms/step - loss: 1.3846 - accuracy: 0.4643
Epoch 9/200
6/6 [==============================] - 0s 3ms/step - loss: 1.4055 - accuracy: 0.4286
Epoch 10/200
6/6 [==============================] - 0s 3ms/step - loss: 1.3498 - accuracy: 0.4286
Epoch 11/200
6/6 [=======================